In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import uuid
from os.path import join
from math import ceil, floor, pi

/home/settnozz/tz_zpoken/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Functions for Augmentation

In [2]:
IMAGE_SIZE = 28


def add_gaussian_noise(X_imgs):
    gaussian_noise_imgs = []
    row, col, _ = X_imgs[0].shape
    # Gaussian distribution parameters
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    
    for X_img in X_imgs:
        gaussian = np.random.random((row, col, 1)).astype(np.float32)
        gaussian = np.concatenate((gaussian, gaussian, gaussian), axis = 2)
        gaussian_img = cv2.addWeighted(X_img, 0.75, 0.25 * gaussian, 0.25, 0)
        gaussian_noise_imgs.append(gaussian_img)
    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype = np.float32)
    return gaussian_noise_imgs

def rotate_images(X_imgs, start_angle, end_angle, n_images):
    X_rotate = []
    iterate_at = (end_angle - start_angle) / (n_images - 1)
    
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (None, IMAGE_SIZE, IMAGE_SIZE, 3))
    radian = tf.placeholder(tf.float32, shape = (len(X_imgs)))
    tf_img = tf.contrib.image.rotate(X, radian)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
    
        for index in range(n_images):
            degrees_angle = start_angle + index * iterate_at
            radian_value = degrees_angle * pi / 180  # Convert to radian
            radian_arr = [radian_value] * len(X_imgs)
            rotated_imgs = sess.run(tf_img, feed_dict = {X: X_imgs, radian: radian_arr})
            X_rotate.extend(rotated_imgs)

    X_rotate = np.array(X_rotate, dtype = np.float32)
    return X_rotate


def get_translate_parameters(index):
    if index == 0: # Translate left 20 percent
        offset = np.array([0.0, 0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20 percent
        offset = np.array([0.0, -0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20 percent
        offset = np.array([0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.8 * IMAGE_SIZE)) 
    else: # Translate bottom 20 percent
        offset = np.array([-0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        h_end = IMAGE_SIZE 
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    offsets = np.zeros((len(X_imgs), 2), dtype = np.float32)
    n_translations = 4
    X_translated_arr = []
    
    tf.reset_default_graph()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(n_translations):
            X_translated = np.zeros((len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 3), 
                dtype = np.float32)
            X_translated.fill(1.0) # Filling background color
            base_offset, size, w_start, w_end, h_start, h_end = get_translate_parameters(i)
            offsets[:, :] = base_offset 
            glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
            
            glimpses = sess.run(glimpses)
            X_translated[:, h_start: h_start + size[0], \
             w_start: w_start + size[1], :] = glimpses
            X_translated_arr.extend(X_translated)
    X_translated_arr = np.array(X_translated_arr, dtype = np.float32)
    return X_translated_arr


def tf_resize_images(X_img_file_paths):
    X_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, (None, None, 3))
    tf_img = tf.image.resize_images(X, (IMAGE_SIZE, IMAGE_SIZE), 
                                    tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # Each image is resized individually as different image may be of different size.
        for index, file_path in enumerate(X_img_file_paths):
            img = mpimg.imread(file_path)[:, :, :3] # Do not read alpha channel.
            resized_img = sess.run(tf_img, feed_dict = {X: img})
            X_data.append(resized_img)

    X_data = np.array(X_data, dtype = np.float32) # Convert to numpy
    return X_data


def central_scale_images(X_imgs, scales):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype = np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype = np.float32)
    box_ind = np.zeros((len(scales)), dtype = np.int32)
    crop_size = np.array([IMAGE_SIZE, IMAGE_SIZE], dtype = np.int32)
    
    X_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (1, IMAGE_SIZE, IMAGE_SIZE, 3))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for img_data in X_imgs:
            batch_img = np.expand_dims(img_data, axis = 0)
            scaled_imgs = sess.run(tf_img, feed_dict = {X: batch_img})
            X_scale_data.extend(scaled_imgs)
    
    X_scale_data = np.array(X_scale_data, dtype = np.float32)
    return X_scale_data

# Simple CNN

In [3]:
learning_rate = 0.001
num_steps = 3000
batch_size = 128

num_input = 28
num_classes = 9
dropout = 0.25


def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        
        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)

    return out


def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

# Helpful functions for process data etc.

In [4]:
def preprocessing(test_bool):
    if test_bool:
        dataset_folder = os.getcwd() + '/dataset/test/'  # '/home/settnozz/tz_zpoken/dataset/test/'
    else:
        dataset_folder = os.getcwd() + '/dataset/train/' # '/home/settnozz/tz_zpoken/dataset/train/'
    labels = sorted(os.listdir(dataset_folder))  # ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII']
    list_of_classes_paths = [join(dataset_folder, i) for i in labels]
    for i in list_of_classes_paths:
        save_images(i, central_scale_images(generate_list_images_array(i), [0.90, 0.80, 0.75]))
        save_images(i, translate_images(generate_list_images_array(i)))
        save_images(i, rotate_images(generate_list_images_array(i), -15, 15, 10))
        save_images(i, add_gaussian_noise(generate_list_images_array(i)))
        print(f'Images for {i} class, is ready')

def get_all_images_array(test_bool, nums, number_of_images):
    data_image = []
    if test_bool:
        dataset_folder = os.getcwd() + '/dataset/test/'  # '/home/settnozz/tz_zpoken/dataset/test/'
    else:
        dataset_folder = os.getcwd() + '/dataset/train/' # '/home/settnozz/tz_zpoken/dataset/train/'
    labels = sorted(os.listdir(dataset_folder))
    list_of_classes_paths = [join(dataset_folder, i) for i in labels]
    paths_to_classes = [join(dataset_folder, i) for i in labels]
    x = [join(i, j) for i in paths_to_classes for j in os.listdir(i)]
    for i in x:
        image = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
        data_image.append((image - image.mean()) / (image.std() + 1e-8))
    return np.array(data_image, dtype='float32'), np.array(sorted(nums * number_of_images), dtype='uint8')


def generate_list_images_array(path_class):
    file_list = sorted([path_class + '/' + i for i in os.listdir(path_class)])
    return tf_resize_images(file_list)

def save_images(path_to_class, arr_of_img):
    for i in arr_of_img:
        cv2.imwrite(path_to_class + '/' + str(uuid.uuid4()) + '.jpg', i)

# Data augmentation for all training images

In [5]:
preprocessing(False)

Images for /home/settnozz/tz_zpoken/dataset/train/I class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/II class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/III class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/IV class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/V class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/VI class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/VII class, is ready
Images for /home/settnozz/tz_zpoken/dataset/train/VIII class, is ready


In [14]:
X, Y = get_all_images_array(False, [1,2,3,4,5,6,7,8], 1760)

In [7]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvgki49fy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f70d8a66c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training model

In [8]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X, y=np.array(Y),
    batch_size=batch_size, num_epochs=None, shuffle=True)
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpvgki49fy/model.ckpt.
INFO:tensorflow:loss = 2.2789931, step = 1
INFO:tensorflow:global_step/sec: 9.56456
INFO:tensorflow:loss = 2.3426502, step = 101 (10.457 sec)
INFO:tensorflow:global_step/sec: 9.6161
INFO:tensorflow:loss = 0.53809285, step = 201 (10.399 sec)
INFO:tensorflow:global_step/sec: 8.77297
INFO:tensorflow:loss = 2.3115249, step = 301 (11.399 sec)
INFO:tensorflow:global_step/sec: 3.09897
INFO:tensorflow:loss = 1.2084901, step = 401 (32.270 sec)
INFO:tensorflow:global_step/sec: 4.7798
INFO:tensorflow:loss = 0.45703697, step = 501 (20.921 sec)
INFO:tensorflow:global_step/sec: 4.6612
INFO:tensorflow:loss = 0.17225441, step = 601 (21.454 sec)
INFO:tensorflow:global_step/sec: 5.34838
INFO:tenso

# Data augmentation for all testing images

In [9]:
preprocessing(True)

Images for /home/settnozz/tz_zpoken/dataset/test/I class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/II class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/III class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/IV class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/V class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/VI class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/VII class, is ready
Images for /home/settnozz/tz_zpoken/dataset/test/VIII class, is ready


In [10]:
X, Y = get_all_images_array(True, [1,2,3,4,5,6,7,8], 440)

# Testing model

In [17]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X, y=Y,
    batch_size=batch_size, shuffle=False)

model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-21-18:55:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvgki49fy/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-21-18:55:37
INFO:tensorflow:Saving dict for global step 3000: accuracy = 1.0, global_step = 3000, loss = 0.0025802986


{'accuracy': 1.0, 'loss': 0.0025802986, 'global_step': 3000}

Task has been done. Accuracy 99+%